Kasviryhmän luokittelu 
===================

Tässä harjoituksessa tarkoituksena on soveltaa koneoppimista kirjastojen esimerkkiaineistojen sijaan oikeaan kerättyyn aineistoon. 

**Ongelma**: Voiko kasviryhmän tunnistaa kasvulohkojen metatiedoista?

## Data lataus
Lataa Pandas DataFrame-olioina aineistot "../../ml-datasets/peltolohkot.csv" ja "../../ml-datasets/kasvikoodit.csv".

Lataa vain 10% [otos](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) koko datasta

In [ ]:
import pandas as pd
from IPython.display import display, HTML


parcels = pd.read_csv("../../ml-datasets/peltolohkot.csv", index_col=False).sample(frac=0.1)
codes = pd.read_csv("../../ml-datasets/kasvikoodit.csv", index_col="code")

print("Lohkodata – koko: ", len(parcels))
display(parcels.head(3))
print("Kasvikoodidata – koko:", len(codes))
display(HTML(codes.head(5).to_html()))

## Datan esikäsittely

Muodosta aluksi kasvikoodidatasta sanakirja, jossa kasvikoodi on avaimena ja ryhmä on arvona ([vinkki](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_dict.html)). 

In [ ]:
codes_dict = codes.to_dict()['group']
print(codes_dict)

Sitten luo lohkodataan uusi sarake `group`, ja sijoita siihen aluksi kasvikoodi (`KVI_KASVIKOODI`) ja sen perusteella ryhmä  ([vinkki](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html)). 
Valitse tämän jälkeen vain rivit, joissa ryhmä on määritelty (!=-1).

In [ ]:
parcels['group'] = parcels['KVI_KASVIKOODI'].fillna(-1.0).astype(int)
parcels.group = parcels.group.map(codes_dict)

parcels.group = parcels.group.fillna(-1)
parcels = parcels[parcels.group != -1]
parcels.head(3)

Tarkastelemalla yllä olevaa tulostetta huomaat, että `KASVULOHKOTUNNUS` ei ole numeerinen arvo. Ennen luokittelua se on syytä muuttaa numeeriseksi. Yksi tapa tehdä tämä on muuttaa. [Jätetään](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) samalla pois alkuperäinen kasvikoodi `KVI_KASVIKOODI` ja lajikekoodi `KLE_LAJIKEKOODI`.

In [ ]:
parcel_tunnus_dict = {i: x for x, i in enumerate(sorted(parcels['KASVULOHKOTUNNUS'].unique()))}
print(parcel_tunnus_dict)

In [ ]:
parcels_modified = parcels.copy().drop(['KVI_KASVIKOODI', 'KLE_LAJIKEKOODI'], axis=1)
parcels_modified['KASVULOHKOTUNNUS'] =  parcels_modified['KASVULOHKOTUNNUS'].map(parcel_tunnus_dict)
parcels_modified.head(3)

#### Muuttujien valinta

Muodosta nyt havaintojoukko `X` ja luokkajoukko `y`.
Valitse havaintojoukkoon mieleikkäämmät sarakkeet lohkodatasta `parcels_modified`. Saat valittua vain osan sarakkeista alla olevalla tyylillä:
```python
# Olemassa olevat
print(list(df.columns))
df = df[['vain', 'halutut', 'sarakkeet']]
```

Valitse luokiksi `y` kasviryhmäsarake `group`.

In [ ]:
print(list(parcels_modified.columns))
# Poista kommenttimerkki '#' haluamasi muuttujien edestä 
X = parcels_modified[[
 #'X',
 #'Y',
 #'fid',
 #'VUOSI',
 #'KASVULOHKOTUNNUS',
 #'LUOMUVKD_KOODI',
 #'SIELAKD_KOODI',
 #'PINTAALA',
 #'ONKOEKOLOGINENALA',
 #'LISATIETO',
 #'JATTOPVM',
 #'PERPVM',
 #'PAIPVM',
]]
y = parcels_modified.group

#### Jako
Jaa luomasi X ja luokat y opetus- ja testijoukkoihin X_train, X_test, y_train ja  y_test siten, että testijoukon osuus on 80% ja opetusjoukon osuus on 20% havainnoista.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5)

##### Datan visualisointia
Visualisoidaan opetusjoukkoa hieman ongelman hahmottamiseksi. Koska valitsemiesi muuttujien määrä voi olla enemmän kuin 2, käytetään PCA-analyysiä visualisoinnin tukena. 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
pca = PCA(n_components=2)
X_train_scaled = StandardScaler().fit_transform(X_train)
X_train_pca = pca.fit_transform(X_train_scaled)
print("Muuttujien määrä oli ennen {}, nyt se on {}".format(len(X.columns), len(X_train_pca[0])))

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap=plt.cm.Set1,
            edgecolor='k')
ax.set_xlabel('1. ominaisuusvektori')
ax.set_ylabel('2. ominaisuusvektori')
plt.show()

## Luokittelu
Muodosta sitten putki ([`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline)), johon liität StandardScaler ja PCA esikäsittelyvaiheet, käytä PCA:n `n_components`-parametrina arvoa 0.7. Lisää putken viimeiseksi komponentiksi haluamasi luokittelija nimelle "clf". Jos käytät Keras-luokittelijaa, muista luoda funktio sille oletusparametreilla.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

pipe = Pipeline([('scaler', StandardScaler()), ('pca',PCA(n_components=0.7)), ('clf',SVC())])

Suorita sitten hyperparametrioptimointi. Putkitusta käytettäessä laita parametrisanakirjaan luokittelijakomponentin nimi ja kaksi alaviivaa ennen parametrin nimeä. Esimerkiksi SVM-algoritmin `C`-parametrin tapauksessa:
```python
params_without_pipeline = {'C':[1,10]}
params_with_pipeline = {'clf__C':[1,10]}

```

In [ ]:
from sklearn.model_selection import GridSearchCV

def train(pipe, parameters):
    clf = GridSearchCV(pipe, parameters, cv=2, n_jobs=-1)
    clf.fit(X_train, y_train)
    print("Parhaat parametrit: ", clf.best_params_)
    print("Paras opetus OA: {:.4f}".format(clf.best_score_))
    print("OA: {:.4f}".format(clf.score(X_test, y_test)))
    return clf

In [ ]:
clf = train(pipe, {'clf__C':[1], 'clf__gamma':[1]})
y_pred = clf.predict(X_test)

### Evaluointi
Overall Accuracy vihjaisi mallin olevan erittäin hyvä! Tarkista kuitenkin vielä luokitteluraportin ja sekaannusmatriisin avulla mallin hyvyys.

In [ ]:
# Evaluoi
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from utils import plot_confusion_matrix

def evaluate(y_pred):
    names =  ['1 - Harkapapu','2 - Herne','3 - Juurikkaat',
              '4 - Kesanto','5 - Kevatrapsi','6 - Kevatviljat',
              '7 - Nurmi','8 - Peruna','9 - Rypsi','10 - Syysvilja']
    
    cm = confusion_matrix(y_test, y_pred)
    print(classification_report(y_test, y_pred))
    plot_confusion_matrix(cm, list(range(1,11)), names, normalize=True)

In [ ]:
evaluate(y_pred)

## Ongelman korjaamista
Yllä olevaa sekaannusmatriisia tarkastelemalla saatat huomata, että melkein kaikki testijoukon havainnot on luokiteltu kuuluvan kehteen luokkaan 6 tai 7. Tarkastele seuraavaksi kuinka paljon havaintoja on missäkin luokassa opetusjoukossa.

In [ ]:
from collections import Counter, OrderedDict
distribution = dict(OrderedDict(sorted(Counter(y_train).items(), key=lambda t: t[0])))
distribution

Havaintojen epätasapaino vaikuttaa olevan ongelma. Sen korjaamiseksi kokeile [RandomUnderSampler](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.under_sampling.RandomUnderSampler.html#imblearn.under_sampling.RandomUnderSampler)-luokkaa ja [Pipeline_imb](https://imbalanced-learn.readthedocs.io/en/stable/api.html#module-imblearn.pipeline)-luokkaa putkittamiseen. Muodosta putki samaan tapaan kuin edellä, mutta lisää RandomUnderSampler ennen luokittelijaa. Nyt voit myös asettaa PCA-parametrin `n_components` korkeammaksi, sillä suorituskyky on resamplauksen vuoksi parempi.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as Pipeline_imb

pipe_imb = Pipeline_imb([('scaler', StandardScaler()), 
                         ('pca',PCA(n_components=0.9)), 
                         ('rus', RandomUnderSampler('not minority')),
                         ('clf',SVC())])

##### Korjauksen jälkeinen evaluointi
Tämän jälkeen suorita hyperparametrien optimointi ja evaluointi.

In [ ]:
clf = train(pipe_imb, {'clf__C':[1, 10], 'clf__gamma':[0.1, 0.01, 1]})
y_pred = clf.predict(X_test)
evaluate(y_pred)

Kokeile vielä valita eri määrä muuttujia kohdassa [Muuttujien valinta](#Muuttujien-valinta). Aja se solu ja tämän jälkeen solut [Jako](#Jako) ja [Korjauksen jälkeinen evaluointi](#Korjauksen-jälkeinen-evaluointi).